### Bitcoin block sampler
[Bitcoin core RPC API](https://en.bitcoin.it/wiki/Original_Bitcoin_client/API_calls_list)

#### Operation ordering


In [1]:
# For secrets
from secret import rpc_user, rpc_password

In [2]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is running!')

Bitcoind PID: 25029



In [3]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [4]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}')
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

Best Block Heights: 619294, Time: 2020-02-28T10:53:24+09:00
CPU times: user 13.6 ms, sys: 680 µs, total: 14.3 ms
Wall time: 50.2 ms


In [ ]:
%%time
sheight = float('inf')
eheight = 0

start = datetime.datetime(2018, 1, 1, 0, 0, 0, tzinfo=tz_seoul)
end = datetime.datetime(2018, 12, 31, 23, 59, 59, tzinfo=tz_seoul)

bi = 10000
for i in range(bi, best_block["height"], 10000):
    commands = [ ['getblockhash', height] for height in range(bi, i) ]
    block_hashes = rpc_connection.batch_(commands)
    commands = [ ['getblock', h] for h in block_hashes ]
    blocks = rpc_connection.batch_(commands)
    ntx = [ block['nTx'] for block in blocks ]

    for block in blocks:
        if get_time(block['time']) < start:
            continue
        elif get_time(block['time']) > end:
            continue
        if sheight > block['height']:
            sheight = block['height']
        if eheight < block['height']:
            eheight = block['height']
    bi = i
    print(f'Reading {i} / {best_block["height"]} ({i/best_block["height"]})', end='\r')
print(f'Query-reply done for {tcnt} transactions')